# Bandits Class

<img src="figures/oab.jpg" style="width: 400px;"/>


# Motivating Example

**Question**: Which route should you take to commute ?

<table>
<tr>
<td><img src="figures/motiv1.png"/></td>
<td><img src="figures/motiv3.png"/></td>
<td><img src="figures/motiv2.png"/></td>
</tr>
</table>



- **Problem**: each day we obtain a _limited feedback_ (traveling time of the chosen route)
- **Remarks**: repeatedly taking the same route 
    - increases our confidence on its true value...
    - but potentially wastes your time !
- **Strategy**: balance learning and optimization (aka Explore/Exploit)

# Class overview
- The problem
    - difference with supervised learning
- Applications
- Performance metric
- Classical Algorithms (stochastic bandits)
    - E-greedy
    - UCB
    - Thompson Sampling
- Extensions
    - adversarial bandits
    - contextual bandits
- more generally, Reinforcement Learning

# The problem

## Difference with supervised learning

- **supervised learning**: 
    - model $P(Y|X)$
    - for each data point in the training set you have access to the true label
- **bandit**: 
    - you have to *choose an action* (aka "pull an arm") to get feedback
    - and you're not going to have feedback on unpulled arms

# Applications

- Medecine
    - test different variants of a drug with unknown power and side effects
    - save as many patients as possible !
- Packet routing
    - choose different routes for a packet
    - optimize for latency
- News recommendations
    - a single slot on homepage, many possible news
    - optimize for clicks, dwell time etc
- Online advertising
    - an opportunity to display an ad, multiple ads available
    - optimize for clicks, sales etc

## Bandits at Criteo

Each time a display is won, we can choose a specific creative.

Suppose we come up with a list of products, how to choose the best creative ?

The layout is composed in real time, choosing different parameters:
- call to action
- color set
- interactivity
- ...

The search space is huge: $\prod\limits_{p}^{}{card(p)}$

<table>
<tr>
<td><img src="figures/darwin1.png" style="width:150px"/></td>
<td><img src="figures/darwin3.png" style="width:150px"/></td>
<td><img src="figures/darwin2.png" style="width:150px"/></td>
</tr>
</table>

# The Bandit Game

- The learner has $i=1,\ldots,K$ arms
- At each round t = 1, . . . , n
    - At the same time
        - The environment chooses a vector of rewards $\{X_{i,t}\}$
        - The learner chooses an arm $I_{t}$
    - The learner receives a reward $X_{i,t}$
    - The environment does not reveal the rewards of the other arms $X_{j,t}, j \ne i$

# Performance metric: regret

$$R_n(\mathcal{A}) = \max_{i=1,\ldots,K}\mathbf{E}[ \sum_{t=1}^n{} X_{i,t} ] - \mathbf{E}[ \sum_{t=1}^n{} X_{I_t,t} ]$$

where $\mathbf{E}[.]$ takes into account any source of randomness

> Basically, the expected reward difference between "always pulling the best arm" and "pulling an arm according to algorithm $\mathcal{A}$"

- Since the environment doesn't give info on unpulled arms, the learner must pull many arms to gain confidence on $X_{i,}$ 
- But pulling a non-optimal arm induces regret...

The learner should solve two contradictory objectives !

**<center>Explore vs Exploit</center>**

# Regret (stochastic bandits)

Suppose we are in a stochastic environment: rewards are i.id draws from probability distributions with a given mean $\mu_i$.

If we introduce $T_{i,n} = \sum_{t=1}^{N} \mathbf{1}[I_{t} = i]$ the number of pulls of arm $i$ at time $n$ we can rewrite regret as:

$$
\begin{align}
R_n(\mathcal{A}) = n \mu_i^{*} - \mathbf{E}[ \sum_{t=1}^n{} X_{I_t,t} ] \\
= \sum_{i \ne i^*} \mathbf{E}[T_{i,n}](\mu_i^* - \mu_i) 
= \sum_{i \ne i^*} \mathbf{E}[T_{i,n}]\Delta_i
\end{align}
$$

With $\Delta_i$ the mean difference between the best arm and the chosen arm $i$

> in other words: we can focus on how many times on average the algo (aka policy) pulls non-optimal arms

**Remark**: if we had access to all rewards at time $t$, the optimal strategy would be to play $\underset{i}{\operatorname{argmax}} \bar{X_{i,t}}$ where $\bar{X_{i,t}} = \frac{1}{n}\sum_{t=1}^{n}X_{i,t}$

Instead, we only have access to $\hat{X_{i,t}} = \frac{1}{|\{I_t = i\}|}\sum_{t=1}^{n}X_{i,t}\mathbf{1}[I_t = i]$

# E-Greedy

- Algorithm:
    - at each time step, choose randomly
        - either an explore action, with probability $\epsilon$
        - or an exploit action with probability $1 - \epsilon$
    - explore = choose a random arm, uniformly
    - exploit = choose best arm so far: $\underset{i}{\operatorname{argmax}} \hat{X_{i,t}}$ 

> $\epsilon$ controls the explore/exploit tradeoff


# Example

- 5 Bernoulli arms with $p = (.1, .1, .1, .1, .9)$

<img src="figures/egreedy1.png" style="width:500px">

# E-greedy analysis

- simple, lower bound on regret:
    - with proba at least $\epsilon$, a suboptimal arm is chosen $(K-1)/K$ times
    - so $\mathbf{E_t}[R] > \epsilon \frac{K-1}{K} \min_i{\Delta_i}$

- in our 5 Bernoulli arms example:
    - lower bound on regret: $\mathbf{E_t}[R] > \epsilon \times (4/5) \times (.9 - .1) = .64 \epsilon$
    - how to choose $\epsilon$ ? 

| $\epsilon$ | strategy | $R>$ |
|-----------|----------|--------------------------------|
| .1 | exploit >> explore | .064 |
| .5 | exploit = explore | .320 |

Note: pay attention to the time horizon: if $max(t)=50$, $\epsilon=.2$ is better

# Beyond E-greedy

- Weaknesses
    - choosing the optimal $\epsilon$ is non trivial (depends on horizon)
    - after some time:
        - we should have confidence on good vs bad arms
        - but the exploration budget is still the same for both (does not depend on $\hat{\Delta_i}$)
- Still, 
    - capable of adapting to non-stationary distributions at the cost of a constant exploration
    - ultra-simple to implement

- Improvement path:
    - adapt the explore/exploit ratio over time
    - $\Rightarrow$ need to estimate the confidence of $\hat{X_i,t}$ or $\hat{\Delta_i}$

# Transition : estimation of means

It appears that choosing the best arm boils down to estimating the mean reward of each arm $\mu_i$ based on the experience we have pulling this arm $\hat{X_{i,t}}$

There are many statistics results one can use to derive bounds. 

A simple, non-parametric one is Hoeffding's concentration inequality:

$$\forall{t,i,\epsilon > 0} \quad P(\hat{X_{i,t}} - \mu_i > \epsilon) < exp(-2t\epsilon^2), $$

> The probability to make an error of more than $\epsilon$ decays exponentially with the number of samples $t$

Corollary: the number of mistakes can be bounded

# UCB

- So we can use H. inequality to build a confidence interval of $\mu_i$ using $\hat{X_{i,t}}$ and $|\{I_t = i\}|$

- Strategy:
    - instead of playing arm with maximum $\hat{X_{i,t}}$ (which may be biased), play arm with maximum potential
    - i.e. given a confidence interval on $\hat{X_{i,t}}$, play the Upper Confidence Bound (UCB)

<img src="figures/ucb_ci.png" style="width:400px" />


# UCB


- Algorithm:
    - play $\underset{i}{\operatorname{argmax}} \hat{X_{i,t}} + \sqrt{\frac{2 log(t)}{|\{I_t = i\}|}}$ 
    - = f(knowledge + uncertainty)

# UCB regret analysis
- H. inequality can be used to derive worst case performance: 
$$\mathbf{E}[R_n] < O(\frac{K log(n)}{\Delta})$$
    - with $\Delta = \underset{i}{\operatorname{min}}(\mu_i^* - \mu_i)$
    - $= f(\Delta)$ $\Rightarrow$ the algorithm adapts to the specific reward distribution



- Worst regret possible ?
    - $\underset{\Delta}{\operatorname{sup}} R_n(\Delta) \Rightarrow \Delta = 1/\sqrt{n}$
<img src="figures/ucb_worst.png" style="width:450px" />


# UCB regret analysis

- compare with blindly playing $\operatorname{argmax} \hat{X_{i,t}}$: $\mathbf{E}[R_n] < O(n)$

> We now have a policy with logarithmic regret 

In [ ]:
ns = np.array([10**2, 10**3, 10**4, 10**5, 10**6, 10**7])
deltas = 1/np.sqrt(ns)
plt.semilogx(ns, deltas, label="worst delta")
plt.semilogx(ns, np.log(ns/deltas)/ns, label="$\mathbf{E}[R]$")
plt.xlabel('n = #turns')
#plt.ylabel('worse $\Delta$')
plt.title('Worst $\Delta$ and regret for UCB')
plt.legend()
plt.savefig('figures/ucb_worst.png')

- Extensions
    - parametrized UCB when knowing the time horizon: knowledge + $\rho$ uncertainty
    - using parametric, yet tighter bounds (e.g. Bernstein, Martingale)
    - regret bound as a function of the KL-divergence in proba. of $\mu_i^*$ and $\mu_i$
    - Explore Then Commit policy

# Beyond UCB

- better practical performance ?
- other learning paradigms ?

# Thompson Sampling

- Bayesian model
    - true parameter $\theta^*$ governing the reward process $P(r|I,\theta^*)$
    - ideally, play $\underset{i}{\operatorname{argmax}} \mathbf{E}[r_i|I_i,\theta^{*}]$
- Estimation
    - given, at any time $t$:
        - past observations $D_t=\{(I, r)\}_{\tau \lt t}$
        - the parametric likelihood function $P(r_t|I_t, \theta)$
        - a prior $P(\theta)$
    - we can compute with Bayes theorem
        - posterior distribution: $P(\theta|D_t) \propto \prod_t P(r_t|I_t,\theta) P(\theta)$
- Strategy: probability matching heuristic
    - choose action randomly according to its probabilty to be optimal
        - draw $\theta_t$ according to $P(\theta|D_t)$
        - observe reward, then update $P(\theta|D_t)$

# TS for Bernoulli bandits

- Model
    - parameters: $\theta=(S,F)$: success/failure counters
    - realization probability: $P(r|\theta) \sim Beta(\theta)$
        - note that $\mathbf{E}[Beta(S,F)] = \frac{S}{S+F}$
- Algorithm
    - start with $S = F = 1$ (uniform prior)
    - at each time step $t$
        - draw $\theta_t$ according to $P(r|\theta)$
        - pull arm $\underset{i}{\operatorname{argmax}}P(r_i|\theta_t)$
        - observe $r_{i,t}$, increment $S$ (if $r>0$) or $F$ (if $r=0$)
        - update $P_t(r|\theta) \sim Beta(S_t, F_t)$

<img src="figures/beta_update.png"/>

In [ ]:
%pylab inline
import scipy.stats
import matplotlib.cm as cm

x=np.linspace(0, 1, 1000)
ax = plt.subplot()
colors = cm.autumn(np.linspace(0, 1, 6))[::-1]
#ax.plot(x, scipy.stats.beta.pdf(x, .5, .5), label='$\Theta=(.5,.5$)', color=colors[0])
ax.plot(x, scipy.stats.beta.pdf(x, 1, 1), label='$\Theta=(1,1$)', color=colors[1])
ax.plot(x, scipy.stats.beta.pdf(x, 1, 10), label='$\Theta=(1,10$)', color=colors[2])
ax.plot(x, scipy.stats.beta.pdf(x, 10, 100), label='$\Theta=(10,100$)', color=colors[3])
ax.plot(x, scipy.stats.beta.pdf(x, 100, 1000), label='$\Theta=(100,1000$)', color=colors[4])
ax.plot([.1, .1],[0, 50],'g--', label='$\Theta^*$')
plt.legend()
plt.title('Beta model update for $\Theta^* = .1$')
plt.savefig('figures/beta_update.png')

# Comparison: TS vs UCB
- Synthetic data generated by K Bernoulli arms with $\theta^* = (.5+\epsilon, .5, \ldots, .5)$
<img src="figures/thompsonvsucb.png" style="width:800px"/>

# What's next ?

## Adversarial bandits
- adversarial environment 
    - e.g. spam filter: defender can react to attacker strategy (learn to filter specific spams)
- rewards are no more i.i.d., but chosen by the environment
    
$$R_n(\mathcal{A}) = \max_{i=1,\ldots,K}\sum_{t=1}^n{} X_{i,t} - \sum_{t=1}^n{} X_{I_t,t}$$

- The benchmark is still the best arm but it's not stochastic anymore
- Goal: achieve decent (sub-linear) regret vs any adversary
    - in particular against any $X_{i,t}$ sequence


# EXP3

- Exponential-weight algorithm for Exploration and Exploitation
- Algorithm
    - parameter: $\eta$ (learning rate)
    - initialize arm weights $w_{i,0} = 1$
    - at each time step $t$
        - compute $W_{t-1} = \sum\limits_{i=1}^K w_{i, t-1}$ *(a normalizing constant)*
        - compute $\hat{p}_{i,t} = \frac{w_{i, t-1}}{W_{t-1}}$ *(proba. to choose action)*
        - draw an arm randomly according to $\hat{p}_{i,y}$
        - estimate importance of reward: $\tilde{X}_{i,t} = \frac{X_{i,t}}{{t\cdot\hat{p}_{i}}}$ 
            - reward, weighted by proba. to choose action (importance sampling estimate, unbiased)
            - i.e. the reward is weighted by its surprise potential
        - update $w_{i, t} = w_{i, t-1} exp(\eta \tilde{X}_{i,t})$
            - i.e. exponential update rule


- Note: in practice add an ounce of extra exploration... (inc. another parameter to optimize)

# Simulation

<img src="figures/exp3_run.png" style="width:600px"/>

# EXP3 regret analysis

- in short
    - $\mathbf{E}[R_n] \le O(\sqrt{n K log K})$
- in comparison, online learning with full monitoring achieves
    - $\mathbf{E}[R_n] \le O(\sqrt{n log K})$
    - we loose a factor of $K$ due to bandit monitoring (1 vs $K$ feedbacks at each turn)

# What's next (cont'd) ?

## Contextual bandits

- You have access to additional information: a context, at the beginning of every turn
    - closer to reality (e.g. user variables in case of ad selection)
    - You should now act according to $P(r_{i,t}|I_{i,t}, c_t)$
    
- Remark
    - context-aware policies have potentially more to learn: $\sim O(K \times |C|)$
    - a simple solution is to learn a bandit per context
    - $\Rightarrow$ needs much more exploration

- Solution path
    - possible strategies
        - reduce dimensionality of contexts (back to small nber of arms scenario)
        - learn a (simple) model of reward as a function of context, e.g. $r(a_i,c_i) = <(a_i,c_i) ; \alpha^*>$
    - algorithms in the spirit of what we've seen have been proposed
        - [Contextual Thompson Sampling](http://www.jmlr.org/proceedings/papers/v28/agrawal13.pdf)
        - [EXP4](http://www.jmlr.org/proceedings/papers/v15/beygelzimer11a/beygelzimer11a.pdf)
        - [LinUCB](http://www.jmlr.org/proceedings/papers/v15/chu11a/chu11a.pdf)
- Still an active research area 

# Link with reinforcement learning


- Imagine a bandit with a state...
    - e.g. in the packet routing case
        - learner chooses which route to send packet to destination
        - depending on chosen route there is a chance that next turn you gain/loose access to different routes
    - e.g. in the ad selection case
        - depending on chosen ad
            - new ads are available / old ads disappear
            - users change behavior (hence the reward function changes)

Overall, a much richer/powerfull/challenging learning problem

> A bandit is merely a Markov Decision Process with 1 state

# Take aways

- Learning *with limited feedback*
- Tradeoff information vs performance


- Available policies for
    - stochastic: log(n) regret
    - adverserial: sqrt(n) regret


- Links with online/reinforcement learning

# Acknowledgements

## Material used in the class
This class borrows material from the following resources:

- Courses
    - [A. Lazaric: Advanced topics in ML (slides)](http://researchers.lille.inria.fr/~lazaric/Webpage/Home/Entries/2012/1/31_Course_on_%22Advanced_topics_of_machine_learning_theory_and_online_learning%22_files/poli-bandit.pdf)
    - [S. Katariya: Bandits intro (slides)](http://homepages.cae.wisc.edu/~sumeet/files/banditsslides.pdf)
    - [V. Perchet: (course + slides)](https://sites.google.com/site/vianneyperchet/)
- Papers
    - [Chapelle et al., An Empirical Evaluation of Thompson Sampling](https://papers.nips.cc/paper/4321-an-empirical-evaluation-of-thompson-sampling)
    - [Auer et al., UCB revisited](http://personal.unileoben.ac.at/rortner/Pubs/UCBRev.pdf)
- Blogs
    - [Kun: EXP3 proof](https://jeremykun.com/2013/11/08/adversarial-bandits-and-the-exp3-algorithm/)
    - [Langford: Hunch.net](http://hunch.net/?p=298)
    - [Szepesvari: Banditalgs](http://banditalgs.com/) a nice blog that'll end up as a book
    - [Bubeck: Bandits state of the art](https://blogs.princeton.edu/imabandit/2016/05/13/bandit-theory-part-ii/)

## Additional readings

If interested, I encourage you to read
- Olivier's paper (for empirical evaluation & applications) 
- Vianney's course (for detail and spirit of the proofs)

and perhaps to subscribe to Szepesvari's blog :)